In [ ]:
%matplotlib inline

import cv2
import numpy as np
import scipy

from tqdm.notebook import tqdm

import seaborn as sns

import matplotlib.pyplot as plt

import utils
import disparity_functions

In [ ]:
data_ix = 1
if data_ix == 0:
    img_list = [cv2.imread("dataset/data_disparity_estimation/Plastic/view1.png"),
                cv2.imread("dataset/data_disparity_estimation/Plastic/view5.png")]

elif data_ix == 1:
    img_list = [cv2.imread("dataset/data_disparity_estimation/Cloth1/view1.png"),
                cv2.imread("dataset/data_disparity_estimation/Cloth1/view5.png")]

In [ ]:
fig = plt.figure(figsize=(8*len(img_list), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_list)):
    plt.subplot(1, len(img_list), i+1)
    plt.imshow(cv2.cvtColor(img_list[i], cv2.COLOR_BGR2RGB))

In [ ]:
keypoints, descriptors, img_keypoints = utils.find_keypoints(img_list)
fig = plt.figure(figsize=(8*len(img_keypoints), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_keypoints)):
    plt.subplot(1, len(img_keypoints), i+1)
    plt.imshow(cv2.cvtColor(img_keypoints[i], cv2.COLOR_BGR2RGB))

In [ ]:
matched_points, _ = utils.find_matches(descriptors, use_nndr=False, number_of_matches=50)

filtered_keypoints = []
filtered_keypoints.append([keypoints[0][match[0]] for match in matched_points])
filtered_keypoints.append([keypoints[1][match[1]] for match in matched_points])
img_keypoints = []
img_keypoints.append(cv2.drawKeypoints(img_list[0], filtered_keypoints[0], None))
img_keypoints.append(cv2.drawKeypoints(img_list[1], filtered_keypoints[1], None))

fig = plt.figure(figsize=(8*len(img_keypoints), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_keypoints)):
    plt.subplot(1, len(img_keypoints), i+1)
    plt.imshow(cv2.cvtColor(img_keypoints[i], cv2.COLOR_BGR2RGB))

In [ ]:
(_, mask) = cv2.findHomography(np.float32([kp.pt for kp in filtered_keypoints[1]]), 
                               np.float32([kp.pt for kp in filtered_keypoints[0]]), 
                               cv2.RANSAC, ransacReprojThreshold=3.0)

In [ ]:
good_matches = []
good_points_l = []
good_points_r = []

for i in range(len(mask)):
    if mask[i] == 1:
        good_points_l.append(filtered_keypoints[0][i])
        good_points_r.append(filtered_keypoints[1][i])
        
good_matches.append(good_points_l)
good_matches.append(good_points_r)

matches1to2 = [cv2.DMatch(i, i, 0) for i in range(len(good_matches[0]))]

matching_img = cv2.drawMatches(img_list[0], filtered_keypoints[0], img_list[1], filtered_keypoints[1], matches1to2, None)
fig = plt.figure(figsize=(16, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(matching_img, cv2.COLOR_BGR2RGB))

In [ ]:
left_img_rectified, right_img_rectified = disparity_functions.rectify_images(img_list, filtered_keypoints)
fig = plt.figure(figsize=(8*2, 8))
fig.patch.set_facecolor('white')
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(left_img_rectified, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(right_img_rectified, cv2.COLOR_BGR2RGB))

In [ ]:
img_list_r = [cv2.resize(img, (0,0), fx=0.5, fy=0.5) for img in img_list]
img_l = img_list_r[0]
img_r = img_list_r[1]
keypoints_r, descriptors_l, descriptors_r, max_i, max_j = disparity_functions.compute_descriptors(img_l, img_r)

In [ ]:
disp_img = np.zeros((img_l.shape[0], img_l.shape[1]))
for i in tqdm(range(img_l.shape[1])):
    for j in range(img_l.shape[0]):
        matched_point = disparity_functions.match_point(keypoints_r, descriptors_l, descriptors_r, (i, j), 40, max_i, max_j)
        disp_img[j][i] = np.sum(np.abs(np.subtract(matched_point, (i, j))))

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.resize(disp_img, (0,0), fx=2.0, fy=2.0, interpolation=cv2.INTER_NEAREST), cmap='gray')

In [ ]:
cv2.imwrite(f"output/disparity_{data_ix}_1.png", cv2.resize((((disp_img - np.min(disp_img)) / (np.max(disp_img) - np.min(disp_img))) * 255).astype(np.uint8),
                                                            (0,0), fx=2.0, fy=2.0, interpolation=cv2.INTER_NEAREST))

In [ ]:
disp_img = np.zeros((img_l.shape[0], img_l.shape[1]))
for i in tqdm(range(img_l.shape[1])):
    for j in range(img_l.shape[0]):
        matched_point = disparity_functions.match_point(keypoints_r, descriptors_l, descriptors_r, (i, j), 40, max_i, max_j, compute_right_img=True)
        disp_img[j][i] = np.sum(np.abs(np.subtract(matched_point, (i, j))))

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.resize(disp_img, (0,0), fx=2.0, fy=2.0, interpolation=cv2.INTER_NEAREST), cmap='gray')

In [ ]:
cv2.imwrite(f"output/disparity_{data_ix}_2.png", cv2.resize((((disp_img - np.min(disp_img)) / (np.max(disp_img) - np.min(disp_img))) * 255).astype(np.uint8),
                                                            (0,0), fx=2.0, fy=2.0, interpolation=cv2.INTER_NEAREST))